# Our Goal: Merging and data cleansing all Leap In ! files 

## Files Used:
    1. Category_codes.csv
    2. Invoices.csv
    3. Member_details.csv
    4. plan_budgets.csv
    5. Postcodes.csv
    6. SupportItemMap.csv

# Summary of Process: How are we merging ? 

* Step 1. Merging tables <b><i>Invoices</i> </b>   and <b><i>Category_codes</i> </b>   on "l3_id" as "category_invoice".
* Step 2: Preparing <i><b> Plan_budgets </b></i> to merge with <i> <b> category_invoice </b></i>
* Step 3: Preparing <i><b> category_invoice</b></i> to merge with <i> <b>plan_budget_agg</b></i> with 3 conditions provided by Industry Advisor.
    1. condition 1: category_invoice.start_date >= Plan_budgets.start_date & category_invoice.end_date <= Plan_budgets.end_date
    2. condition 2: category_invoice.membership_number == Plan_budgets.membership_number
    3. condition 3: category_invoice.l2_key == Plan_budgets.l2_key
* Step 4: Merging Table <b><i> Member_details </i></b> and <b><i> category_invoice_budget</i></b> on column <b><i> membership_number </i> </b>
* Step 5: Merging <b><i>SupportItemMap</i></b> table to <b><i>merged table till step 4</i></b>  on "l3_reference_number"column.
* Step 6: Merging <b><i>Postcodes</i></b> and <b><i>merged table till step 5</i></b> on column "Id" and create a final merged table named as <b><i>leap_in_df</i></b>
* Step 7: Creating <b>"spending Behaviours"</b> column on table <b><i>leap_in_df</i></b> and finally creating <b><i>leap_in_40pc_df</i></b>
                                                                                                 

In [544]:
# Imporing requied library files
import pandas as pd
import numpy as np
import statistics as stat

In [545]:
# path of files located local drive. 
file_path = "/Users/ajaymoktan/Desktop/IFN711/Leap In Data/Budget utilisation/Private data/"


In [546]:
# Importing all the six files from local Drive as a pands' Dataframe.
Category_codes = pd.read_csv(file_path + "Category_codes.csv")
Invoices = pd.read_csv(file_path + "Invoices.csv")
Member_details = pd.read_csv(file_path + "Member_details.csv")
plan_budgets = pd.read_csv(file_path + "plan_budgets.csv")
Postcodes = pd.read_csv(file_path + "Postcodes.csv")
SupportItemMap = pd.read_csv(file_path + "SupportItemMap.csv")


# Step 1. Merging tables "Invoices" and "Category_codes" on "l3_id" as "category_invoice".

In [547]:
# Renaming the column from 'Category_3_id' to  'l3_id'
Invoices.rename(columns= {'Category_3_id': 'l3_id'}, inplace =True)

In [548]:
category_invoice =  pd.merge(Invoices, Category_codes, \
                                         how='left',\
                                         on='l3_id')

#  Step 2: Preprating  Plan_budgets table.

* 2.1 Slice "Plan_budgets" as "plan_budgets_l3_key" that contains l3_key with value.
* 2.2 Create "plan_budget" with l3_key with no values. It means we are not inluding category l3 funded amount. 
* 2.3 checking the duplicate rows in "plan_budget" based on 'plan_key', 'l2_key' and drop them.
* 2.4 plan_budget grouped by "plan_key" as "plan_budget_group".
* 2.5 plan_budget_group is aggregiated as "plan_budget_agg" and sum the allocation and rest select first. 


## Step 2.1: Slicing "plan_budgets" to extract dataframe containg non null l3_keys only as "plan_budgets_l3_key"

In [549]:
# sorting plan_budgets in descending order based on "plan_id" column 
# so that we get the latest (correct) values first

plan_budgets.sort_values(by=['plan_id'], inplace=True, ascending=False)
# plan_budgets.head()

In [550]:
# plan_budgets_l3_key containing observations that has l3_key
plan_budgets_l3_key = plan_budgets[~plan_budgets['l3_key'].isnull()]

## Step 2.2: Slicing "plan_budgets" to extract dataframe containg Null l3_keys only as "plan_budget" (As per business logic provided)

In [551]:
# dataframe "plan_budget" contains observation that has NULL values as l3_key.
plan_budget = plan_budgets[plan_budgets['l3_key'].isnull()]

In [552]:
plan_budget.head(2)

,plan_id,plan_key,plan_start_date,plan_end_date,member_key,allocation,l2_key,l3_key,membership_number
19312,101502,e60f7e00-c179-11ea-8daf-e7e742291ebd,2020-04-01,2021-04-07,d9d68660-c179-11ea-8a11-397bdc21f5a9,0.0,7d0af52b-f597-11e9-bfb4-022d4762bb3c,NaN,11810329
19311,101501,e60f7e00-c179-11ea-8daf-e7e742291ebd,2020-04-01,2021-04-07,d9d68660-c179-11ea-8a11-397bdc21f5a9,0.0,7d0af53b-f597-11e9-bfb4-022d4762bb3c,NaN,11810329


## Step 2.3: Chekcing the Duplicate rows in "plan_budget" table based on 'plan_key', 'l2_key' and dropping them 

In [553]:
# checking duplicate based on 'plan_key', 'l2_key'
plan_budget.duplicated(subset=['plan_key', 'l2_key']).sum()

21461

In [554]:
# Dropping the duplicate values found.
plan_budget = plan_budget.drop_duplicates(subset=['plan_key', 'l2_key'], keep = 'first')

In [555]:
# plan_budget.head()

## Step 2.4: Now the "plan_budget" is grouped by column "plan_key"

In [556]:
# Dropping column l3_key 
plan_budget.drop('l3_key', axis=1, inplace=True)


In [557]:
# grouping the plan_budget table on plan_key
plan_budget_group = plan_budget.groupby('plan_key')

In [558]:
# plan_budget_group.apply(print)

## Step 2.5: Aggregating plan_budget_group as "plan_budget_agg" to calcuate sum of allocation and only take first row of rest of the columns.

In [559]:
# Aggregating the "plan_budget_group" and naming as "plan_budget_agg"
plan_budget_agg = plan_budget_group.agg({'plan_id': 'first', 'plan_start_date': 'first', 'plan_end_date': 'first', 'member_key': 'first', 'allocation': 'sum', 'l2_key'\
                      : 'first', 'membership_number': 'first'})



In [560]:
# Converting "plan_budget_agg" to pandas dataframe
plan_budget_agg.reset_index(inplace=True)

#checking the output
plan_budget_agg.head(2)

,plan_key,plan_id,plan_start_date,plan_end_date,member_key,allocation,l2_key,membership_number
0,00001ea0-d23c-11ea-ad9d-c9ee3068ede7,23374,2020-03-31,2020-08-17,fc567790-d23b-11ea-a13a-d188ddbdc773,47445.55,7d0af53b-f597-11e9-bfb4-022d4762bb3c,314530517
1,0003fb30-c7f4-11ea-b247-07c902e1140e,5218,2019-12-17,2020-10-07,f63dff60-c7f3-11ea-baa6-ebfe92aca975,236423.38,7d0af57b-f597-11e9-bfb4-022d4762bb3c,984933966


# Step 3: Preparing table "category_invoice" to merge with "plan_budget_agg" wtih the following 3 conditions:

* <i><b> condition 1:</b> </i> Condition 1 should be category_invoice.start_date >= plan_buget_agg.start_date & category_invoice.end_date <= plan_budget_agg.end_date

* <i><b>condition 2:</b></i> category_invoice.membership_number == plan_budget_agg.membership_number

* <i><b>condition 3:</b></i> category_invoice.l2_key == plan_budget_agg.l2_key

    * Step 3.1: Using condition 2 and 3, merging the tables category_invoice and plan_budget_agg on l2_key and membership_number as "category_invoice_budget". 
    * Step 3.2: Applying condition 1 to filter out the dataframe "category_invoice_budget" 




### Converting date into tamestamp to compare dates

In [561]:
# converting to datatime datatype
plan_budget_agg[['plan_start_date', 'plan_end_date']] = plan_budget_agg[['plan_start_date', 'plan_end_date']].apply(pd.to_datetime)
category_invoice[['start_date', 'end_date']] = category_invoice[['start_date', 'end_date']].apply(pd.to_datetime)



#### Step 3.1: Using condition 2 and 3, merging the tables category_invoice and plan_budget_agg on l2_key and membership_number

In [562]:
# Merging the table using condition 2 and 3. 
category_invoice_budget =  pd.merge(category_invoice, plan_budget_agg, \
                                         how='inner',\
                                         on=['membership_number','l2_key'])


## Step 3.2: Applying condition 1 to filter out the dataframe "category_invoice_budget"

In [563]:
# filter_condtion = condition 1
filter_condition = (category_invoice_budget.start_date >= category_invoice_budget.plan_start_date) & \
(category_invoice_budget.end_date <= category_invoice_budget.plan_end_date)

# Applying filter_condtion on the category_invoice_budget
category_invoice_budget = category_invoice_budget.loc[filter_condition]

In [564]:
# Checking the shape of the table.
category_invoice_budget.shape

(146026, 24)

# Step 4: Merging Table "Member_details " and "category_invoice_budget " on column " membership_number" as "category_invoice_budget_member".

* <b>Step 4.1 </b> From "category_invoice_budget_member" it is found that the members which has member_exit_date (206 members) but only 22 members were exited before the plan ends and they were removed from "category_invoice_budget_member" table. "category_invoice_budget_member_cleaned" table is created after removing exited members. 



### Converting column name 'NDIS_Number' to  'membership_number' of  "Member_details " table

In [565]:
# Renaming the column name from'NDIS_Number'to 'membership_number'as they mean same. 
Member_details.rename(columns= {'NDIS_Number': 'membership_number'}, inplace =True)

In [566]:
# Merging the tables category_invoice_budget and  Member_details
category_invoice_budget_member = pd.merge(category_invoice_budget, Member_details, \
                                         how = 'inner', on = 'membership_number')


In [567]:
print(category_invoice_budget.shape, category_invoice_budget_member.shape)

(146026, 24) (145967, 28)


## Step 4.1: Preparing "category_invoice_budget_member" table:
* <b> Dropping all the rows associated with the members who have exited before the end of plan. 

In [568]:
# subset of the dataset "category_invoice_budget_member" with member exit date. 

member_exit_df = category_invoice_budget_member[['membership_number', 'plan_start_date', 'plan_end_date', 'Member_exit_date']]


In [569]:
# from member_exit_df extracting only those rows that has memberexit date in the column "member_exit_date"
member_exit = member_exit_df[~member_exit_df['Member_exit_date'].isnull()]
member_exit

,membership_number,plan_start_date,plan_end_date,Member_exit_date
0,484394812,2019-08-15,2020-07-15,26/11/2020 0:00
1,484394812,2019-08-15,2020-07-15,26/11/2020 0:00
2,484394812,2019-08-15,2020-07-15,26/11/2020 0:00
3,484394812,2019-08-15,2020-07-15,26/11/2020 0:00
4,484394812,2019-08-15,2020-07-15,26/11/2020 0:00
...,...,...,...,...
144567,182493529,2020-10-01,2020-11-26,26/3/2021 0:00
144595,96661094,2020-09-22,2020-12-21,28/1/2021 0:00
144596,96661094,2020-09-22,2020-12-21,28/1/2021 0:00
144597,96661094,2020-09-22,2020-12-21,28/1/2021 0:00


In [570]:
# converting datatype to datetime datatype of Member_exit_date column.
member_exit['Member_exit_date'] = member_exit['Member_exit_date'].apply(pd.to_datetime)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [571]:
# grouping the member_exit table by 'membership_number' to fine the membership number
member_exit_group = member_exit.groupby('membership_number')

In [572]:
# member_exit_group.get_group(993412941)

In [573]:
# member_exit_group is aggregiated and named as member_exit_agg
member_exit_agg = member_exit_group.agg({'plan_start_date':'first', 'plan_end_date':'first', 'Member_exit_date':'first'})





In [574]:
# creating member_exited dataframe.
member_exited = member_exit_agg.reset_index()

In [575]:
#finding out Members who exited before plan ended on "member_exited" table
#which means: Member_exit_date < plan_end_date
member_exit1 = member_exited.loc[member_exited.Member_exit_date < member_exited.plan_end_date]

In [576]:
member_exit1.shape

(22, 4)

In [577]:
# Finally finding out the list of membership numbers who exited before plan ended. 
member_exited_list = member_exit1.membership_number
member_exited_list

0       12171458
9       44373247
17      78177238
20      81951320
32     143165743
60     293600120
67     328360347
69     329626850
70     336279820
75     376167104
83     398872647
90     427747947
105    503921371
110    519724712
133    632942145
140    680067331
142    693001963
143    699619325
145    701058767
166    795741481
172    832612550
202    961495989
Name: membership_number, dtype: int64

####  Hence, "member_exited_list" is the list of members (22 members ) who exited before the plan ended and we will remove all the records of those members from main table "category_invoice_budget_member" in the following steps. 

In [578]:
# Keeping only the record of members that are not in the member exited list. 
category_invoice_budget_member_cleaned = \
category_invoice_budget_member[~category_invoice_budget_member['membership_number'].isin(member_exited)]

In [579]:
# checking the shape of the table so far. 
category_invoice_budget_member_cleaned.shape

(145967, 28)

In [580]:
# Since we do not need columns 'plan_id'and 'Member_exit_date' we are dropping them here. 
category_invoice_budget_member_cleaned = \
category_invoice_budget_member_cleaned.drop(columns = {'plan_id', 'Member_exit_date'})

In [581]:
category_invoice_budget_member_cleaned.shape

(145967, 26)

# Step 5: Merging SupportItemMap table to "category_invoice_budget_member_cleaned table on "l3_reference_number".

* <b>Step 5.1: </b> Getting rid of frequency column with labels "Short-term" and "One-off" from SupportItemMap(as per business logic provided)

* <b>step 5.2:</b> "category_invoice_budget_member_support" is created after merging.


In [582]:
SupportItemMap.head(2)

,reference_number,name,uom,registration_group,frequency
0,01_001_0101_1_1,Transitional Support,EA,Accommodation / Tenancy Assistance,Short-term
1,05_090000155_0103_1_2,AT Rental Products for Personal Care and Safety,EA,Assist Prod-Pers Care/Safety,Either


####  Renaming reference_number to l3_reference_number on table SupportItemMap

In [583]:
# Renaming the column 'reference_number'to'l3_reference_number'
SupportItemMap.rename(columns={'reference_number':'l3_reference_number'}, inplace = True)

### Step 5.1: Getting rid of frequency column with labels "Short-term" and "One-off" 

In [584]:
SupportItemMap1 = SupportItemMap[(SupportItemMap['frequency']!= "Short-term")]

In [585]:
support_item = SupportItemMap1[SupportItemMap1['frequency']!= "One-off"]

### Step 5.2: Support_item with only "Recurring" and "Either" is ready to be merged with table "category_invoice_budget_member_cleaned" on column "l3_reference_number"

In [586]:
# Merging the tables category_invoice_budget_member_cleaned and support_item on column l3_reference_number
category_invoice_budget_member_support = pd.merge(category_invoice_budget_member_cleaned, support_item,\
                                                 how = 'inner',\
                                                 on = 'l3_reference_number')

In [587]:
# checking the shape of the column
category_invoice_budget_member_support.shape

(86621, 30)

# Step 6: Merging table "Postcodes" and "category_invoice_budget_member_support" on column "Id" as "leap_in_df"

In [588]:
# Merging the tables category_invoice_budget_member_support and Postcodes on column Id and 
# Creating the final merged file name as leap_in_df. 
leap_in_df = pd.merge(category_invoice_budget_member_support, Postcodes,\
                     how = 'left',
                     on = 'Id')

In [589]:
# checking the shape of the final merged file. 
leap_in_df.shape

(88314, 31)

# Step 7 : Creating spending Behaviours column on table "leap_in_df"

## Steps for calculating Sepnding Behaviours:
#### <i>7.1 : calculate plan period ( plan_end_date - plan_start_date ) and add as a new column as "plan_period".</i>
#### <i>7.2: Dataframe is groupby 'plan_key','l2_key','membership_number' and 'frequency' columns. </i>
#### <i>7.3: Aggregiate the groupby-dataframe and convert it into dataframe.</i>
#### <i>7.4: Columns used to calculate Spending behaviours are: "funded_amount", "allocation", "plan_period_days", "frequency". </i>
#### <i>7.5: As per Business logic provided: </i>
#### <i>7.5.1:under "frequency" column, Recurring = Regular and Either = Episodic </i>
#### <i>7.5.2: dollar value spent = totalfunded_amount, dollar value available  = allocation - total funded_mount, plan period = plan_period_days, plan progress = number of days since plan start </i>
#### <i> Since there is a variance in plan_period ( starting form 0 plan period) the ratio of (plan progress in days / plan period ) = 0.4 is agreed to be proceed with. which is 40% through the total plan period to calculate plan progress date which in turn is used to determine plan progress. </i> 


### <i>7.1 : calculate plan period ( plan_end_date - plan_start_date ) and add as a new column as "plan_period".</i>

In [590]:
# calculating plan_period 
leap_in_df['plan_period'] = (leap_in_df['plan_end_date']- leap_in_df['plan_start_date'])

#converting plan_period into int16. 

leap_in_df['plan_period'] = leap_in_df['plan_period'].dt.days.astype('int16')

### 7.1.1 Dropping all the rows with plan_period < 30 days as adviced by Industry Supervisor.

In [591]:
# keeping only those members/plans that have plan period of 7 and greater. 
leap_in_df.drop(leap_in_df[leap_in_df['plan_period'] < 30].index,inplace = True)
leap_in_df.shape

(87924, 32)

### 7.1.2 create new column "plan_progress_40pc" which is (0.4 * plan_period) 

In [592]:
# calculate 40% of the plan period and converting into integer value to avoid decial values. 
leap_in_df['plan_progress_40pc'] = leap_in_df['plan_period']*0.4

# converting into datatype datatype.
leap_in_df['plan_progress_40pc'] =leap_in_df['plan_progress_40pc'].astype('int16')

### 7.1.3 create new column "plan_progress_40pc_date" which is ( plan_progress_40pc+ plan_start_date) 

In [593]:
# creating column "plan_progress_40pc_date" by adding plan_start_date and 40% of the plan_progress days 
# which is in the column plan_progress_40pc column. 

leap_in_df['plan_progress_40pc_date'] = \
[plan_start_date + pd.Timedelta(days=pc40_day) \
for plan_start_date, pc40_day in zip(leap_in_df['plan_start_date'],leap_in_df['plan_progress_40pc'])]



### 7.2 Groupby "leap_in_df" on 'plan_key','l2_key','membership_number' and 'frequency' columns as "leap_in_df_group"

In [594]:
leap_in_df_group = leap_in_df.groupby(['plan_key','l2_key','membership_number','frequency'])

#### checking each group with 'plan_key','l2_key','membership_number' and 'frequency' for manually varification purpose

In [595]:
# group1 = leap_in_df_group.get_group(('00111a90-c7f4-11ea-b247-07c902e1140e',\
#                                      '7d0af53b-f597-11e9-bfb4-022d4762bb3c',14541707,"Recurring"))\
# [['plan_start_date','plan_end_date']]  

# group1 = leap_in_df_group.get_group(('65396330-dce6-11ea-8cfb-59022ff97600',\
#                                      '7d0af5b1-f597-11e9-bfb4-022d4762bb3c',192127105,"Either"))\
# [['plan_start_date','plan_end_date']]   

# group1 = leap_in_df_group.get_group(('b4f85630-b462-11ea-a898-85942aaecf99',\
#                                      '7d0af57b-f597-11e9-bfb4-022d4762bb3c',832922816,"Either"))\
# [['plan_start_date','plan_end_date']]   

# group1 = leap_in_df_group.get_group(('b49c5840-d216-11ea-803a-6d3641cbf65d',\
#                                      '7d0af57b-f597-11e9-bfb4-022d4762bb3c',93782210,"Either"))\
# [['plan_start_date','plan_end_date','plan_period']] 
# group1                          

### 7.2.1 Sum only those funded_amount where start_date <= plan_progress_40pc_date on grouped dataset "leap_in_df_group"

In [596]:
# Sum only those funded_amount column in each group based on the condition. which is adding only those invoice 
# amount which has occured till the 40% plan date. 
funded_amount_40pc = leap_in_df_group.apply(lambda group: group[group['start_date']<=group['plan_progress_40pc_date']]\
                                       ['funded_amount'].sum())


In [597]:
# Processing the group series to extract a "total_funded_amount_40pc" series that contains sum of 
# funded amount only. 

funded_df =pd.DataFrame(funded_amount_40pc)
total_funded_df =funded_df.reset_index()

#Renaming the last column as "total_funded_amount_40pc"

total_funded_df.rename(columns={total_funded_df.columns[4]:"total_funded_amount_40pc"}, inplace =True)

### 7.2.2 Aggregiate the "leap_in_df_group" and add the new column "total_funded_amount_40pc"

In [598]:
leap_in_40pc_df_agg = leap_in_df_group.agg({'start_date' : 'first', 'end_date': 'first', 'funded_amount': 'first', 
        'units': 'first', 'l3_id': 'first',
       'invoice_number': 'first', 'invoice_date': 'first', 'abn': 'first',
       'l3_reference_number': 'first', 'l3_key': 'first', 'l3_name': 'first', 'l3_description': 'first',
       'l2_reference_number': 'first', 'l2_name': 'first', 'l1_name': 'first', 'l1_id': 'first',
        'plan_start_date': 'first', 'plan_end_date': 'first', 'member_key': 'first',
       'allocation': 'first', 'Id': 'first', 'YOB': 'first', 'Disability': 'first', 'name': 'first', 'uom': 'first',
       'registration_group': 'first', 'Postcode': 'first', 'plan_period': 'first',
       'plan_progress_40pc': 'first', 'plan_progress_40pc_date': 'first'})

In [599]:
# Converting leap_in_40pc_df_agg into pandas dataframe. 
leap_in_40pc_agg_df = leap_in_40pc_df_agg.reset_index()

### 7.2.3 : finally merge " total_funded_df " and "leap_in_40pc_agg_df" and rearranging the columns as "leap_in_40pc_df".

In [600]:
leap_in_40pc_df = pd.merge(leap_in_40pc_agg_df, total_funded_df, how = 'inner', \
                  on =['plan_key', 'l2_key','membership_number', 'frequency'])

In [601]:
# Rearranging the columns just for convenient purpose.
rearrange_columns_name = ['membership_number','total_funded_amount_40pc','allocation','plan_period','plan_progress_40pc',
                     'plan_progress_40pc_date', 'plan_start_date', 'plan_end_date','plan_key','l2_key','frequency', 'start_date',
                       'end_date', 'funded_amount', 'units', 'l3_id', 'invoice_number',
                       'invoice_date', 'abn', 'l3_reference_number', 'l3_key', 'l3_name',
                       'l3_description', 'l2_reference_number', 'l2_name', 'l1_name', 'l1_id','member_key',  'Id',  
                       'YOB', 'Disability', 'name', 'uom', 'registration_group', 'Postcode']

leap_in_40pc_df = leap_in_40pc_df.reindex(columns = rearrange_columns_name)
leap_in_40pc_df.sample(10)


,membership_number,total_funded_amount_40pc,allocation,plan_period,plan_progress_40pc,plan_progress_40pc_date,plan_start_date,plan_end_date,plan_key,l2_key,...,l1_name,l1_id,member_key,Id,YOB,Disability,name,uom,registration_group,Postcode
2682,621981024,100.00,24597.46,320,128,2020-06-13,2020-02-06,2020-12-22,e6309a40-ddb7-11ea-8b25-0fa931eaa230,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,Core,1,dd33bad0-ddb7-11ea-a31a-7d25e47a5c0a,6645,1969.0,"Cerebral palsy, Intellectual disability","Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",4227
2974,945891017,6483.30,94732.05,98,39,2021-01-23,2020-12-15,2021-03-23,f97b219c-c170-456c-88c7-fb0a2df5e93e,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,Core,1,d4ddc6a0-dcef-11ea-a5ac-61abce895864,22067,1999.0,Arthrogryposis,"Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",4519
1360,616406367,0.00,15020.00,143,57,2020-05-23,2020-03-27,2020-08-17,b1af03e0-d238-11ea-8cf8-59259f9ae6fa,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,Capacity Building,3,afb3f730-d238-11ea-aed8-25028c760942,17537,1943.0,"Tourette?s Syndrome, Obsessive Compulsive Diso...",Exercise Physiology,H,Exercise Physiology And Physical Wellbeing Act...,4171
850,614234487,0.00,125701.60,362,144,2019-08-05,2019-03-14,2020-03-10,667a6eb0-dce6-11ea-8cfb-59022ff97600,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,Capacity Building,3,5ddc2a00-dce6-11ea-812e-09cdf4382371,7968,1943.0,None,"Individual Assessment, Therapy And/Or Training...",H,Therapeutic Supports,None
3048,368446000,11146.57,47926.15,360,144,2020-05-05,2019-12-13,2020-12-07,fba461b0-e52b-11ea-abc2-2bb0308caa66,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,Core,1,f1859780-e52b-11ea-aa27-0bababe260e0,7991,1943.0,None,"Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",None
1942,603558207,1858.09,76579.51,358,143,2020-07-25,2020-03-04,2021-02-25,c70fb3c0-d6d3-11ea-b1ee-87504086bf09,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,Core,1,c41cd990-d6d3-11ea-9cd1-f194648955a5,9275,1991.0,"Adhd, , Bipolar","Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",4156
1498,775792749,1210.01,32347.49,363,145,2019-08-13,2019-03-21,2020-03-18,b53c1ab0-d216-11ea-803a-6d3641cbf65d,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,Capacity Building,3,aece71a0-d216-11ea-bb54-b308798eda15,7485,2008.0,None,Exercise Physiology,H,Exercise Physiology And Physical Wellbeing Act...,4570
1547,868501949,1794.20,11306.24,351,140,2019-07-11,2019-02-21,2020-02-07,b659bfb0-d216-11ea-803a-6d3641cbf65d,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,Capacity Building,3,afe5d510-d216-11ea-bb54-b308798eda15,6626,1993.0,Autism spectrum disorder (ASD),"Individual Assessment, Therapy And/Or Training...",H,Therapeutic Supports,2213
1611,245930691,378.56,20268.48,343,137,2019-02-03,2018-09-19,2019-08-28,b8b45aa0-d50d-11ea-a941-f79507f5b8c8,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,Capacity Building,3,afd0cf90-d50d-11ea-8224-836b70d82925,3226,1929.0,Down syndrome,"Individual Assessment, Therapy And/Or Training...",H,Therapeutic Supports,4161
450,355521543,0.00,104320.90,69,27,2020-12-30,2020-12-03,2021-02-10,4c977b74-b1aa-4998-9e13-8d6898e06f34,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,Core,1,669cb5d0-593f-11eb-8206-71a1145ebd06,28400,1992.0,None,"Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",2641


# 7.3 : Creating final column " spending_behaviour " on dataframe "leap_in_40pc_df". 

### leap_in_40pc_df is the data of a member per plan_key, per l2_key and per frequency. 

#### <i>Regular = Recurring and Episodic = Either </i>
#### <i>Total spent for category = total_funded_amount_40pc</i>
#### <i>Ratio spent to plan progress = (total_funded_amount_40pc/allocation)/(plan_progress_40pc/plan_period)</i>



In [602]:
# defining the variables ratio_spent and plan_progress 
ratio_spent = leap_in_40pc_df['total_funded_amount_40pc']/leap_in_40pc_df['allocation']
plan_progress = leap_in_40pc_df['plan_progress_40pc']/leap_in_40pc_df['plan_period']

# code to execute the following conditions with use of above variables.

category_conditions = [
    ((leap_in_40pc_df['frequency']=='Recurring') & (leap_in_40pc_df['total_funded_amount_40pc']== 0)),
    ((leap_in_40pc_df['frequency']=='Recurring') & (leap_in_40pc_df['total_funded_amount_40pc']==leap_in_40pc_df['allocation'])),
    ((leap_in_40pc_df['frequency']=='Recurring') & (ratio_spent/plan_progress > 1.15)),
    ((leap_in_40pc_df['frequency']=='Recurring') & (ratio_spent/plan_progress < 0.85)),
    leap_in_40pc_df['frequency']=='Recurring', 
    ((leap_in_40pc_df['frequency']=='Either') & (leap_in_40pc_df['total_funded_amount_40pc']==leap_in_40pc_df['allocation'])),
    ((leap_in_40pc_df['frequency']=='Either') & (leap_in_40pc_df['total_funded_amount_40pc']==0)),
    leap_in_40pc_df['frequency']=='Either'
]
# Choices in the same order as the conditions
spending_behaviours = [
    'Not Activated', 
    'All Spent(Recurring)', 
    'Over Spent' , 
    'Under Spent', 
    'On Track' ,
    'All Spent(Either)',
    'None Spent', 
    'Some Spent'
]


leap_in_40pc_df['spending_behaviour'] = np.select(category_conditions, spending_behaviours, "Default")

In [603]:
# Printing some random value 
leap_in_40pc_df.sample(5)



,membership_number,total_funded_amount_40pc,allocation,plan_period,plan_progress_40pc,plan_progress_40pc_date,plan_start_date,plan_end_date,plan_key,l2_key,...,l1_id,member_key,Id,YOB,Disability,name,uom,registration_group,Postcode,spending_behaviour
2497,179451880,0.00,38535.30,383,153,2020-04-06,2019-11-05,2020-11-22,e2e0fa60-c179-11ea-8daf-e7e742291ebd,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,1,de2742e0-c179-11ea-8a11-397bdc21f5a9,16516,1924.0,None,"Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",4508,Not Activated
594,647616311,358.52,8781.85,137,54,2019-09-18,2019-07-26,2019-12-10,6298a0a0-dce6-11ea-8cfb-59022ff97600,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,3,5aa716b0-dce6-11ea-812e-09cdf4382371,6785,1921.0,None,"Individual Assessment, Therapy And/Or Training...",H,Therapeutic Supports,4650,Some Spent
788,383206637,850.00,13621.56,365,146,2020-05-12,2019-12-18,2020-12-17,65aa6340-d20b-11ea-9ae5-59f9297fc5f9,7d0af57b-f597-11e9-bfb4-022d4762bb3c,...,3,63677820-d20b-11ea-ae84-0545f282cdcb,5341,1944.0,None,Therapy Assistant - Level 2,H,Therapeutic Supports,None,Some Spent
1369,284215772,4220.30,73622.56,383,153,2020-08-18,2020-03-18,2021-04-05,b20cd3c0-d216-11ea-803a-6d3641cbf65d,7d0af53b-f597-11e9-bfb4-022d4762bb3c,...,1,af908920-d216-11ea-bb54-b308798eda15,7531,1933.0,Schizoaffective disorder,"Access Community, Social And Rec Activities - ...",H,"Participation In Community, Social And Civic A...",3199,Under Spent
1309,805400487,22.00,271923.71,364,145,2020-04-06,2019-11-13,2020-11-11,aacb7b70-d220-11ea-90b0-473ddd03d6bc,7d0af52b-f597-11e9-bfb4-022d4762bb3c,...,1,a6ef0490-d220-11ea-b70a-d17e589b1829,9012,1961.0,None,Specialised Transport To School/Educational Fa...,D,Assistance With Travel/Transport Arrangements,3630,Some Spent


In [604]:
# Checking the shape
leap_in_40pc_df.shape

(3131, 36)

In [605]:
# counting the number of each behaviours.
leap_in_40pc_df.spending_behaviour.value_counts()

Under Spent          1189
Some Spent           1185
None Spent            463
Not Activated         268
Over Spent             14
On Track               11
All Spent(Either)       1
Name: spending_behaviour, dtype: int64

In [606]:
# Exporting the dataframe to csv file to path provided. 
# leap_in_40pc_df.to_csv(r'/Users/ajaymoktan/Desktop/IFN711/Leap in Merg/leap_in_40pc_df.csv')